### import

In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, trange
from math import exp, sqrt, log
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import i0, i1, iv
from numpy import random
from torch.nn.functional import normalize
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torchsummary import summary

### The setup of the dataset

In [18]:
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)


### Define the model of the neural network

In [20]:
class BASICnet(nn.Module):
    def __init__(self, dim, width):
        super(BASICnet, self).__init__()
        self.dim = dim
        self.width = width

        self.fc1 = nn.Linear(self.dim, self.width)
        self.fc2 = nn.Linear(self.width, self.width)
        self.fc3 = nn.Linear(self.width, self.width)
        self.fc4 = nn.Linear(self.width, self.width)

        self.output = nn.Linear(self.width, 1, bias=True)

    def forward(self, x):
        # s = nn.ConstantPad2d(x, (0, self.dim - self.dim))
        y = self.fc1(x)
        y = torch.tanh(y)
        y = self.fc2(y)
        y = torch.tanh(y)
        y = self.fc3(y)
        y = torch.tanh(y)
        y = self.fc4(y)
        y = torch.tanh(y)
        
        output = self.output(y)
        return output

    def assign_value(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                nn.init.constant_(m.bias.data, 0.1)

class TESTNet(nn.Module):
    def __init__(self):
        super(TESTNet, self).__init__()
        self.fc1 = nn.Linear(10, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 1)

    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

### Cauculate equation (Righthand side of the equation, RHS)

### Generate model from the dataset and using montecarlo method to train the model

In [32]:
import random

def calculate_fx(X):
    # Check if there are boundary cases with 1 and -1 in each row
    has_boundary = torch.any((X == 1) | (X == -1), dim=-1)

    # Create an initial fx tensor filled with the regular values
    regular_fx = -160 * (torch.tensor(4 * np.pi, dtype=torch.float32).pow(2)) * torch.prod(torch.sin(4 * np.pi * X), dim=-1)
    
    # Where there are boundary cases, set fx to zero
    fx = torch.where(has_boundary, torch.tensor(0.0, dtype=torch.float32, device=X.device), regular_fx)

    return fx



def data_generator_monte_carlo(Numberofgenpoints, dim, prob_special = 0.1):
    source = torch.randn(size = (Numberofgenpoints, dim), requires_grad=True) # 生成一个大矩阵
    source = normalize(source, p=2)
    radius = torch.rand(size = (Numberofgenpoints, 1))
    radius = torch.pow(torch.rand(size = (Numberofgenpoints, 1)), 1/dim)
    source = source * radius

    # 随即确认替换的行
    num_replace_incol = torch.randint(1, Numberofgenpoints, size=(1,)).item()

    for _ in range(num_replace_incol):
        if random.random() < prob_special:
            special_value = random.choice([1, -1])
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            source[row_index] = torch.full((dim, ), special_value)
        else:
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            col_index = torch.randint(0, dim, size=(1,)).item()
            replace_value = torch.randint(0, 2, size=(1,)).item() * 2 - 1
            source[row_index, col_index] = replace_value

    max_value = torch.max(source)
    min_value = torch.min(source)

    target = calculate_fx(source)
    # set target as the requres_graade true

    if max_value > 1 or min_value < -1:
        raise ValueError
    
    return source, target

已经生成了包括边界条件的值。

In [22]:
def boundary_loss(input):
    return torch.mean(input**2)

def mse_loss(input, target):
    return torch.nn.MSELoss()(input, target)

### Dataset generate

In [48]:
test_gen = 100
test_dim = 10

test_source, test_target = data_generator_monte_carlo(test_gen, test_dim)

## The optimizer and scheduler

In [24]:

Dim = 10
LR = 1e-3
NUM_EPOCHS = 1000
BATCH_SIZE = 1000
loss_list = np.zeros(NUM_EPOCHS)
test_loss = np.zeros(NUM_EPOCHS)
max_value_loss = np.zeros(NUM_EPOCHS)
min_value_loss = np.zeros(NUM_EPOCHS)

net = BASICnet(dim=Dim, width=100)

# Optimizer and scheduler define there
Optimizer = optim.Adam(net.parameters(), lr=LR)
Scheduler = StepLR(Optimizer, step_size=1000, gamma=0.8)

# orignal print
print(sum(p.numel() for p in net.parameters() if p.requires_grad))
# summary(net, (Dim,), device="cuda:0")



31501


In [80]:
x = test_source
x = x.to("cuda:0")
u = net(x).to("cuda:0")
u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
# Calculate the PDE loss
cx = calculate_fx(x)
# pde_residual = u_xx - cx
pde_loss = torch.mean((u_xx.mean(dim=1) - cx)**2)
print(cx.shape)
print(u_xx.shape)
print(pde_loss.shape)

torch.Size([100])
torch.Size([100, 10])
torch.Size([])


In [90]:
# def pde_loss(u_net, x):
#     u = u_net(x)
#     u_x = torch.autograd.grad(u.sum(), x, create_graph=True)[0]
#     pde = u_x - calculate_fx(x)  # Define the PDE equation based on your problem
#     return torch.mean(pde**2)

def pde_loss(net, x):
    u = net(x)
    u_x = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0]
    # Calculate the PDE loss
    cx = calculate_fx(x)
    # pde_residual = u_xx - cx
    pde_loss = torch.mean((u_xx.mean(dim=1) - cx)**2)
    return pde_loss


def boundary_loss(u_net, x):
    u = u_net(x)
    boundary_points = torch.norm(x, dim=1) >= 1.0  # 找到边界点
    boundary_values = u[boundary_points]  # 边界点的模型输出
    
    # 构建目标边界条件，都应该为0
    target_boundary_values = torch.zeros_like(boundary_values)
    
    # 计算边界损失
    boundary_residual = boundary_values - target_boundary_values
    boundary_loss = torch.mean(boundary_residual**2)
    return boundary_loss


In [93]:
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# 定义批次大小和总Epoch数
total_iterations = 10000
# 定义输入数据的维度
dim = 10
# 定义数据生成器的数量
gen_each_round = 100
LR = 1e-3

mse_cost_function = torch.nn.MSELoss() # Mean squared error
# 初始化网络、优化器和学习率调度器
net = TESTNet()
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=LR)
scheduler = StepLR(Optimizer, step_size=1000, gamma=0.8)

for epoch in trange(total_iterations):
    total_loss = 0.0
    optimizer.zero_grad()
    data, target = data_generator_monte_carlo(gen_each_round, dim, prob_special=0.1)
    # data = Variable(torch.from_numpy(data).float(), requires_grad=False).to(device)
    # target = Variable(torch.from_numpy(target).float(), requires_grad=False).to(device)
    data = data.to(device)
    target = target.to(device)
    
    # 前向传播
    out = net(data)
    mse_loss = mse_cost_function(out, target)

    # loss based on PDE
    pde_loss_value = pde_loss(net, data)
    # loss based on boundary
    boundary_loss_value = boundary_loss(net, data)
    total_loss = 0.0001*mse_loss + pde_loss_value + boundary_loss_value
    # 反向传播和优化
    
    total_loss.backward()
    optimizer.step()
    # 累积总损失

    # 更新学习率
    scheduler.step()

    # # 打印当前Epoch的平均损失
    # average_loss = total_loss / gen_each_round
    if epoch % 100 == 0:
        print(f"Iteration {epoch}/{total_iterations}, Total Loss: {total_loss.item():.4f}, MSE Loss: {mse_loss.item():.4f}, PDE Loss: {pde_loss_value.item():.4f}, Boundary Loss: {boundary_loss_value.item():.4f}")





  0%|          | 0/10000 [00:00<?, ?it/s]

/home/reaganwu/miniconda3/envs/pntorch/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/reaganwu/miniconda3/envs/pntorch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
  0%|          | 11/10000 [00:00<01:42, 97.85it/s]

Iteration 0/10000, Total Loss: 74978.9219, MSE Loss: 74973.5156, PDE Loss: 74971.4062, Boundary Loss: 0.0173


  1%|          | 113/10000 [00:01<01:55, 85.55it/s]

Iteration 100/10000, Total Loss: 295227.7812, MSE Loss: 295221.0000, PDE Loss: 295198.1875, Boundary Loss: 0.0487


...

Iteration 2700/10000, Total Loss: 133964.8594, MSE Loss: 133951.0938, PDE Loss: 133951.4688, Boundary Loss: 0.0030


 28%|██▊       | 2820/10000 [00:25<01:05, 109.43it/s]

Iteration 2800/10000, Total Loss: 689698.9375, MSE Loss: 689634.3750, PDE Loss: 689630.0000, Boundary Loss: 0.0047


 29%|██▊       | 2863/10000 [00:25<01:04, 110.68it/s]


KeyboardInterrupt: 

In [ ]:
gfgafg